In [2]:
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor



/home/jiffy/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
titanic = pd.read_csv('train.csv')
titanic.info()

label = 'Survived'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
titanic_train, titanic_test = train_test_split(titanic, test_size=0.2, random_state=42)
titanic_test_labels = titanic_test[label]
titanic_test_nolabel = titanic_test.drop([label], axis=1)
titanic_train_labels = titanic_train[label]
titanic_train_nolabel = titanic_train.drop([label], axis=1)
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
331,332,0,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,C124,S
733,734,0,2,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,NaN,S
382,383,0,3,"Tikkanen, Mr. Juho",male,32.0,0,0,STON/O 2. 3101293,7.9250,NaN,S
704,705,0,3,"Hansen, Mr. Henrik Juul",male,26.0,1,0,350025,7.8542,NaN,S
813,814,0,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,NaN,S


In [7]:
titanic_predictor = TabularPredictor(label=label, path='.').fit(titanic_train)

/home/jiffy/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Beginning AutoGluon training ...
AutoGluon will save models to "./"
AutoGluon Version:  0.7.0
Python Version:     3.10.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Fri, 07 Apr 2023 02:10:43 +0000
Train Data Rows:    712
Train Data Columns: 11
Label Column: Survived
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Fe

In [8]:
titanic_pred = titanic_predictor.predict(titanic_test_nolabel)
titanic_perf = titanic_predictor.evaluate_predictions(y_true=titanic_test_labels, y_pred=titanic_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.8100558659217877
Evaluations on test data:
{
    "accuracy": 0.8100558659217877,
    "balanced_accuracy": 0.7962033462033462,
    "mcc": 0.6046711888296795,
    "f1": 0.7571428571428571,
    "precision": 0.803030303030303,
    "recall": 0.7162162162162162
}


In [15]:
titanic_predictor.leaderboard(titanic_test, silent=True).sort_values('score_val', ascending=False)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
9,WeightedEnsemble_L2,0.810056,0.846154,0.023151,0.015851,2.033688,0.001275,0.000553,0.347280,2,True,14
10,LightGBMXT,0.804469,0.839161,0.010547,0.004581,0.352037,0.010547,0.004581,0.352037,1,True,3
0,NeuralNetTorch,0.837989,0.832168,0.011330,0.010718,1.334372,0.011330,0.010718,1.334372,1,True,12
7,LightGBM,0.810056,0.832168,0.002742,0.003377,0.365734,0.002742,0.003377,0.365734,1,True,4
8,XGBoost,0.810056,0.825175,0.007237,0.003580,0.314437,0.007237,0.003580,0.314437,1,True,11
3,ExtraTreesEntr,0.832402,0.818182,0.039476,0.032297,0.376545,0.039476,0.032297,0.376545,1,True,9
11,CatBoost,0.798883,0.818182,0.003539,0.003343,0.575439,0.003539,0.003343,0.575439,1,True,7
5,NeuralNetFastAI,0.826816,0.811189,0.010395,0.013499,2.027647,0.010395,0.013499,2.027647,1,True,10
1,ExtraTreesGini,0.837989,0.804196,0.037247,0.036544,0.435939,0.037247,0.036544,0.435939,1,True,8
6,LightGBMLarge,0.815642,0.804196,0.006219,0.003412,0.721821,0.006219,0.003412,0.721821,1,True,13


In [16]:
titanic_predictor.get_model_best()

'WeightedEnsemble_L2'

In [18]:
abs(titanic.corr()["Survived"].sort_values(ascending=False))

/tmp/ipykernel_8158/1063583755.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  abs(titanic.corr()["Survived"].sort_values(ascending=False))


Survived       1.000000
Fare           0.257307
Parch          0.081629
PassengerId    0.005007
SibSp          0.035322
Age            0.077221
Pclass         0.338481
Name: Survived, dtype: float64

In [21]:
titanic.drop(['Parch', 'PassengerId', 'SibSp', 'Age'], axis=1, inplace=True)

KeyError: "['Parch', 'PassengerId', 'SibSp', 'Age'] not found in axis"

In [23]:
titanic.corr()

/tmp/ipykernel_8158/2964377706.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  titanic.corr()


,Survived,Pclass,Fare
Survived,1.000000,-0.338481,0.257307
Pclass,-0.338481,1.000000,-0.549500
Fare,0.257307,-0.549500,1.000000


In [24]:
titanic_train, titanic_test = train_test_split(titanic, test_size=0.2, random_state=42)
titanic_test_labels = titanic_test[label]
titanic_test_nolabel = titanic_test.drop([label], axis=1)
titanic_train_labels = titanic_train[label]
titanic_train_nolabel = titanic_train.drop([label], axis=1)
titanic_train.head()

,Survived,Pclass,Name,Sex,Ticket,Fare,Cabin,Embarked
331,0,1,"Partner, Mr. Austen",male,113043,28.5000,C124,S
733,0,2,"Berriman, Mr. William John",male,28425,13.0000,NaN,S
382,0,3,"Tikkanen, Mr. Juho",male,STON/O 2. 3101293,7.9250,NaN,S
704,0,3,"Hansen, Mr. Henrik Juul",male,350025,7.8542,NaN,S
813,0,3,"Andersson, Miss. Ebba Iris Alfrida",female,347082,31.2750,NaN,S


In [25]:
titanic_predictor = TabularPredictor(label=label, path='.').fit(titanic_train)

Beginning AutoGluon training ...
AutoGluon will save models to "./"
AutoGluon Version:  0.7.0
Python Version:     3.10.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Fri, 07 Apr 2023 02:10:43 +0000
Train Data Rows:    712
Train Data Columns: 7
Label Column: Survived
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11208.54 MB
	Train Data (Original)  Memory Usage: 0.23 MB (0.0% of available memory)
	Inferring data type of each feature based on

In [26]:
titanic_pred = titanic_predictor.predict(titanic_test_nolabel)
titanic_perf = titanic_predictor.evaluate_predictions(y_true=titanic_test_labels, y_pred=titanic_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.8156424581005587
Evaluations on test data:
{
    "accuracy": 0.8156424581005587,
    "balanced_accuracy": 0.798970398970399,
    "mcc": 0.6165515965990812,
    "f1": 0.7591240875912408,
    "precision": 0.8253968253968254,
    "recall": 0.7027027027027027
}


In [27]:
titanic_predictor.leaderboard(titanic_test, silent=True).sort_values('score_val', ascending=False)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
2,WeightedEnsemble_L2,0.815642,0.853147,0.015385,0.020260,3.117657,0.001403,0.000556,0.333122,2,True,14
0,CatBoost,0.826816,0.839161,0.003737,0.005574,0.697553,0.003737,0.005574,0.697553,1,True,7
3,LightGBM,0.810056,0.832168,0.002513,0.003463,0.238765,0.002513,0.003463,0.238765,1,True,4
1,XGBoost,0.821229,0.825175,0.006760,0.003563,0.383035,0.006760,0.003563,0.383035,1,True,11
5,LightGBMLarge,0.804469,0.818182,0.004792,0.003345,0.601876,0.004792,0.003345,0.601876,1,True,13
4,NeuralNetTorch,0.810056,0.811189,0.010718,0.009380,1.215998,0.010718,0.009380,1.215998,1,True,12
7,ExtraTreesGini,0.798883,0.811189,0.043509,0.032310,0.406845,0.043509,0.032310,0.406845,1,True,8
10,LightGBMXT,0.770950,0.804196,0.007781,0.003381,0.281226,0.007781,0.003381,0.281226,1,True,3
6,ExtraTreesEntr,0.804469,0.797203,0.038920,0.035332,0.410242,0.038920,0.035332,0.410242,1,True,9
11,NeuralNetFastAI,0.765363,0.797203,0.010245,0.014130,2.086981,0.010245,0.014130,2.086981,1,True,10


In [29]:
titanic_predictor.get_model_best()

'WeightedEnsemble_L2'

In [30]:
titanic_processsed = pd.read_csv('titanic_processed.csv')

In [31]:
titanic_train_processsed, titanic_test_processsed = train_test_split(titanic_processsed, test_size=0.2, random_state=42)
titanic_test_labels_processsed = titanic_test_processsed[label]
titanic_test_nolabel_processsed = titanic_test_processsed.drop([label], axis=1)
titanic_train_labels_processsed = titanic_train_processsed[label]
titanic_train_nolabel_processsed = titanic_train_processsed.drop([label], axis=1)
titanic_train_processsed.head()

,Unnamed: 0,PassengerId,Survived,Pclass,Fare,Sex_Numeric,Embarked_C,Embarked_S,"Fare_Category_(-0.001, 7.854]","Fare_Category_(41.579, 512.329]","Fare_Category_(7.854, 10.5]","Age_Category_(-0.08, 16.0]","Age_Category_(16.0, 32.0]",Pclass_1,Pclass_2,Pclass_3
331,331,332,0.0,1,28.5000,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
733,733,734,0.0,2,13.0000,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
382,382,383,0.0,3,7.9250,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
704,704,705,0.0,3,7.8542,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
813,813,814,0.0,3,31.2750,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [32]:
titanic_predictor_processsed = TabularPredictor(label=label, path='.').fit(titanic_train_processsed)

Beginning AutoGluon training ...
AutoGluon will save models to "./"
AutoGluon Version:  0.7.0
Python Version:     3.10.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Fri, 07 Apr 2023 02:10:43 +0000
Train Data Rows:    712
Train Data Columns: 15
Label Column: Survived
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0.0, 1.0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    10619.29 MB
	Train Data (Original)  Memory Usage: 0.09 MB (0.0% of available memory)
	Inferring data type of each feature bas

In [34]:
titanic_pred_processsed = titanic_predictor_processsed.predict(titanic_test_nolabel_processsed)
titanic_perf_processsed = titanic_predictor_processsed.evaluate_predictions(y_true=titanic_test_labels_processsed, y_pred=titanic_pred_processsed, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.7932960893854749
Evaluations on test data:
{
    "accuracy": 0.7932960893854749,
    "balanced_accuracy": 0.7679536679536679,
    "mcc": 0.5720157989940912,
    "f1": 0.7131782945736433,
    "precision": 0.8363636363636363,
    "recall": 0.6216216216216216
}


In [35]:
titanic_predictor_processsed.leaderboard(titanic_test_processsed, silent=True).sort_values('score_val', ascending=False)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
6,WeightedEnsemble_L2,0.793296,0.818182,0.063209,0.057657,2.113846,0.002453,0.000612,0.333294,2,True,14
3,LightGBMXT,0.810056,0.804196,0.002122,0.001643,0.281653,0.002122,0.001643,0.281653,1,True,3
4,XGBoost,0.804469,0.804196,0.005616,0.002689,0.134384,0.005616,0.002689,0.134384,1,True,11
0,CatBoost,0.815642,0.797203,0.001847,0.001330,0.325201,0.001847,0.001330,0.325201,1,True,7
5,NeuralNetTorch,0.804469,0.797203,0.005726,0.006962,0.825301,0.005726,0.006962,0.825301,1,True,12
1,NeuralNetFastAI,0.815642,0.790210,0.008040,0.005481,0.468670,0.008040,0.005481,0.468670,1,True,10
8,LightGBM,0.782123,0.790210,0.000872,0.001584,0.268307,0.000872,0.001584,0.268307,1,True,4
9,ExtraTreesEntr,0.776536,0.790210,0.038961,0.033055,0.392539,0.038961,0.033055,0.392539,1,True,9
10,ExtraTreesGini,0.776536,0.776224,0.039009,0.034640,0.358608,0.039009,0.034640,0.358608,1,True,8
7,RandomForestGini,0.787709,0.769231,0.039621,0.031789,0.391293,0.039621,0.031789,0.391293,1,True,5


In [36]:
titanic_predictor_processsed.get_model_best()

'WeightedEnsemble_L2'